<a href="https://colab.research.google.com/github/williamokomba/Flower-classification-using-EffientNet-Transfer-learning-RES/blob/main/flower_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flower Image  Classification using EfficientNet Transfer learning

1. load using API token from kaggle
2. visualize the images
3. data augmentation
4. train test split
5. building EfficientNet model
6. prediction and evaluation


About Dataset

Context

This dataset belongs to DPhi Data Sprint #25: Flower Recognition. The dataset contains raw jpeg images of five types of flowers.

* daisy
* dandelion
* rose
* sunflower
* tulip

Content

* train - contains all the images that are to be used for training your model.
  In this folder you will find five folders namely - 'daisy', 'dandelion', 'rose', 'sunflower' and 'tulip' which contain the images of the respective flowers
* test - contains 924 flowers images. For these images you are required to make predictions as the respective flower names - 'daisy', 'dandelion', 'rose', 'sunflower' and 'tulip'
* Testing_set_flower.csv - this is the order of the predictions for each image that is to be submitted on the platform. Make sure the predictions you download are with their image's filename in the same order as given in this file.

In [ ]:
#import dependences
from zipfile import ZipFile
import os
import cv2
#!pip install imgaug
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
#uploading data directly from kaggle
# Create API key
#install kaggle library
#!pip install kaggle


In [ ]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

[flower dataset kaggle link](https://www.kaggle.com/datasets/imsparsh/flowers-dataset)Import flower dataset api

In [ ]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d imsparsh/flowers-dataset

Dataset URL: https://www.kaggle.com/datasets/imsparsh/flowers-dataset
License(s): CC0-1.0
flowers-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#extracting the compresed sentiment dataset
from zipfile import ZipFile
file_name = "flowers-dataset.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
#Rename train to flowwrs and Create it base Directory
#import os
os.rename('train', 'flowers')
datadir = '/content/flowers'
Categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

OSError: [Errno 39] Directory not empty: 'train' -> 'flowers'

In [ ]:
#Display the random images from each folder
for category in Categories:
    path = os.path.join(datadir, category)
    imgs = os.listdir(path)

    fig, axes = plt.subplots(1, 3, figsize=(10, 6))
    fig.suptitle(f'{category}', fontsize= 15)   # Use suptitle to set the figure title

    # Iterate over the 3 random images in each folder
    for i in range(3):
        img_name = imgs[np.random.randint(0, len(imgs))]
        img_path = os.path.join(path, img_name)
        img = plt.imread(img_path)

        #Read the images from the path
        image_array = cv2.imread(img_path)

        #convert BGR to RGB color format
        image_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

        #show image
        axes[i].imshow(image_rgb)
        axes[i].axis('off')

    plt.show()

Image Augmentation

Data augmentation increases the accuracy of the model

In [ ]:
#Creating a function to count images in each folder to ensure that augmentation took place

def count_images(directory):
    for category in Categories:
        path = os.path.join(directory, category)
        num_images = len(os.listdir(path))
        print(f'{category}: {num_images} images')

In [ ]:
#count original images in each category
count_images(datadir)

In [ ]:
#data augmentation
aug = iaa.Sequential([
#horizontal flips
iaa.Fliplr(0.5),
# randomly croping images upto 10% os it's size
iaa.Crop(percent = (0, 0.1)),
# scaling the image between 80% and 120% and rotating it between -25 to 25
iaa.Affine(scale = (0.8, 1.2), rotate = (-25, 25)),
# changing the brightness of the images
iaa.Multiply((0.8, 1.2)),
# adjusting the contrast of the images
iaa.LinearContrast((0.75, 1.5))
])

#create a function to augment the images
def augment_images(directory, Categories):
    for category in Categories:
        path = os.path.join(directory, category)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            augmented_img = aug.augment_image(img)

#create a new file to hold uagmented images and then add them to original folder
            new_filename = os.path.splitext(filename)[0] + '_augmented' + os.path.splitext(filename)[1] # Indent this line to be inside the loop
            cv2.imwrite(os.path.join(path, new_filename), augmented_img) # Indent this line to be inside the loop

#apply the function to all the images
augment_images(datadir, Categories)

In [ ]:
#check the new total images
count_images(datadir)

#### Resizing the images to 224

In [ ]:
#Resizing the images to 224
#Current size
image_array.shape


In [ ]:
#Resize to 224
IMG_SIZE = 224
new_array = cv2.resize(image_array, (IMG_SIZE, IMG_SIZE))
new_array.shape

In [ ]:
#visualize
#change to rgb
new_array = cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB)
#visualize
plt.imshow(new_array);

### Create Training data

In [ ]:
#creating training data and target

training_data = []

def create_training_data():
    for category in Categories:
        path = os.path.join(datadir, category)
        labels = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, labels])
            except Exception as e:
                pass

create_training_data()

In [ ]:
#check training data
len(training_data)